In [1]:
import numpy as np
import glob
import os

In [2]:
# This order is necessary
import cellprofiler
import cellprofiler.preferences
cellprofiler.preferences.set_awt_headless(True)
cellprofiler.preferences.set_headless()
import cellprofiler.modules
import cellprofiler.utilities
import cellprofiler.utilities.cpjvm
cellprofiler.utilities.cpjvm.cp_start_vm()
import cellprofiler as cp

Done


In [3]:
m = cp.modules.exporttodatabase.ExportToDatabase()

In [6]:
m.allow_overwrite.choices

['Never', 'Data only', 'Data and schema']

In [15]:
m.write_properties_file

<bound method ExportToDatabase.write_properties_file of <cellprofiler.modules.exporttodatabase.ExportToDatabase object at 0x7fd14c05b4d0>>

In [21]:
#dir(m)

In [27]:
cp.modules.exporttodatabase.DIR_CUSTOM_WITH_METADATA

'Custom folder with metadata'

In [28]:
cp.modules.exporttospreadsheet.ABSOLUTE_FOLDER_NAME

'Elsewhere...'

In [30]:
#??cp.modules.exporttospreadsheet

In [32]:
m.db_name.value

'DefaultDB'

In [25]:
m.directory.value = cp.preferences.ABSOLUTE_FOLDER_NAME
m.directory.custom_path

'Default Output Folder|'

In [ ]:
m.experiment_name.value = 'Exp'
m.table_prefix.value = 'Exp_'
m.save_cpa_properties.value = True
m.location_object.value = 'Cell'
m.directory.value = cp.preferences.ABSOLUTE_FOLDER_NAME
m.directory.custom_path = '/path/to/dir'
m.db_type.value = cp.modules.exporttodatabase.DB_SQLITE
m.allow_overwrite.value = cp.modules.exporttodatabase.OVERWRITE_ALL

In [113]:
# os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/jre'
#os.getenv('JAVA_HOME')

In [7]:
import cellprofiler.pipeline

# See headless options at: https://github.com/CellProfiler/CellProfiler/blob/master/cellprofiler/__main__.py

In [75]:
pipeline = cellprofiler.pipeline.Pipeline()
pipeline.load('/lab/repos/cytokit/image-quantification-v2.cppipe')

In [76]:
pipeline.modules()

In [ ]:
pipeline.run()

In [7]:
#m = pipeline.modules()[-1]
m = pipeline.modules()[2]

In [8]:
m

In [11]:
sg = m.assignments[1]

In [19]:
sg.rule_filter.value #CONTAINS_PREDICATE.display_name

u'and (file does contain "nucleus_mask")'

In [83]:
#m = cp.modules.exporttospreadsheet.ExportToSpreadsheet()

In [128]:
#m.directory.dir_choice = cp.modules.exporttospreadsheet.ABSOLUTE_FOLDER_NAME # "Elswhere..."
#m.directory.custom_path 

In [135]:
def get_module_images():
    m = cp.modules.images.Images()
    m.filter_choice.value = cellprofiler.modules.images.FILTER_CHOICE_IMAGES
    return m

def get_module_metadata():
    m = cp.modules.metadata.Metadata()
    return m

def get_module_namesandtypes():
    # Resources:
    # - https://github.com/CellProfiler/CellProfiler/blob/master/tests/modules/test_namesandtypes.py#L960
    m = cp.modules.namesandtypes.NamesAndTypes()
    m.assignment_method.value = cp.modules.namesandtypes.ASSIGN_RULES
    # "Image set matching" -- determines how images from different channels are aligned
    # ORDER means that the file paths must sort alphabetically in a way that will align them
    m.matching_choice.value = cp.modules.namesandtypes.MATCH_BY_ORDER
    m.assignments[0].image_name.value = u'Cell'
    m.assignments[0].rule_filter.value = u'file does contain "cell_mask"'
    m.assignments[0].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_OBJECTS
    m.add_assignment()
    m.assignments[1].image_name.value = 'Nucleus'
    m.assignments[1].rule_filter.value = 'file does contain "nucleus_mask"'
    m.assignments[1].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_OBJECTS
    m.add_assignment()
    m.assignments[2].image_name.value = 'CD3'
    m.assignments[2].rule_filter.value = 'file does contain "CD3.tif"'
    m.assignments[2].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_GRAYSCALE_IMAGE
    m.add_assignment()
    m.assignments[3].image_name.value = 'CD8a'
    m.assignments[3].rule_filter.value = 'file does contain "CD8a.tif"'
    m.assignments[3].load_as_choice.value = cp.modules.namesandtypes.LOAD_AS_GRAYSCALE_IMAGE
    return m

def get_module_measureobjectsizeshape():
    m = cp.modules.measureobjectsizeshape.MeasureObjectSizeShape()
    m.object_groups[0].name.value = 'Cell'
    m.add_object()
    m.object_groups[1].name.value = 'Nucleus'
    return m

def get_module_exporttospreadsheet(output_dir):
    m = cp.modules.exporttospreadsheet.ExportToSpreadsheet()
    m.directory.dir_choice = cp.modules.exporttospreadsheet.ABSOLUTE_FOLDER_NAME # "Elswhere..."
    m.directory.custom_path = output_dir
    m.prefix.set_is_yes(True)
    m.prefix.value = 'cpexport_'
    m.delimiter.value = cp.modules.exporttospreadsheet.DELIMITER_TAB
    m.add_metadata.value = True
    m.wants_overwrite_without_warning.value = True
    return m

def get_pipeline():
    pipeline = cp.pipeline.Pipeline()
    
    m = get_module_images()
    m.module_num = 1
    pipeline.add_module(m)
    
    m = get_module_metadata()
    m.module_num = 2
    pipeline.add_module(m)
    
    m = get_module_namesandtypes()
    m.module_num = 3
    pipeline.add_module(m)
    
    m = get_module_measureobjectsizeshape()
    m.module_num = 4
    pipeline.add_module(m)
    
    m = get_module_exporttospreadsheet('/lab/data/tmp/tile_y00_x00/export')
    m.module_num = 5
    pipeline.add_module(m)
    
    return pipeline

In [136]:
pl = get_pipeline()

In [137]:
#p.add_pathnames_to_file_list(['/lab/data/tmp/tile_y00_x00'])

In [138]:
paths = [
    '/lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
    '/lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif',
    '/lab/data/tmp/tile_y00_x00/expression_CD3.tif',
    '/lab/data/tmp/tile_y00_x00/expression_CD8a.tif'
]
paths

['/lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 '/lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif',
 '/lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 '/lab/data/tmp/tile_y00_x00/expression_CD8a.tif']

In [77]:
pipeline.modules()

In [95]:
m = pipeline.modules()[2]
sg = m.assignments[0]

In [96]:
sg.object_name.value, sg.image_name.value

(u'Cell', u'Cell')

In [92]:
sg = m.assignments[2]

In [94]:
sg.object_name.value, sg.image_name.value

(u'Cytoplasm', u'CD3')

In [139]:
pl.add_urls(['file://' + p for p in paths])

In [140]:
pl.file_list

['file:///lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 'file:///lab/data/tmp/tile_y00_x00/expression_CD8a.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif']

In [141]:
measurements = cp.measurement.Measurements()
workspace = cp.workspace.Workspace(pl, m, None, None, measurements, None)
file_list = pl.get_filtered_file_list(workspace)

In [142]:
file_list

['file:///lab/data/tmp/tile_y00_x00/expression_CD3.tif',
 'file:///lab/data/tmp/tile_y00_x00/expression_CD8a.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_cell_mask.tif',
 'file:///lab/data/tmp/tile_y00_x00/object_nucleus_mask.tif']

In [143]:
measurements = pl.run()

In [144]:
measurements.names

['CD3', 'CD8a']

In [145]:
measurements.object_names

[u'Cell', u'Experiment', u'Image', u'Nucleus']

In [82]:
measurements.get_measurement(u'Cell', u'AreaShape_Area')

array([ 477,  566, 2503, 1545, 1331,  910,  366, 1424,  338,  425,  815,
       1042,  867,  688, 1258,  372,  221, 1510, 2411, 1922, 3084, 1457,
       2308,  410, 1501, 2172, 2042, 1326, 1746, 1467, 2233, 1602,  889,
       1114,  529, 2780,  802, 2011, 1424,  733, 1622, 2013, 1390, 2120,
        738, 1973, 1766,  605, 2140, 1219, 1266, 2179, 1028,  697, 1164,
       1006,  538, 1739,  709,  511,  662,  480, 1733, 1258, 1539, 1441,
       1385, 1696, 1670, 1897,  952, 1534, 1442, 1310, 1255, 1269, 1609,
        834, 1390, 1220,  867,  930, 1862,  275, 1010,  785,  483,  545,
       2438, 1479, 1277, 1067, 1446, 1698,  254,  892, 2083, 1470, 1184,
       1385, 1051, 1617, 1264, 1157, 1670, 1248, 1489, 1854, 1188,  574,
        841, 1584, 2140, 2043, 1166,  279, 1440,  927, 2112, 1707, 2155,
       1319, 1327,  527, 1407, 2758,  665, 1208, 1245, 1326,  880,  542,
       1118,  863, 1426, 1300, 1837,  626,  335, 1282, 1888,  689,  409,
       1536,  456,  477, 1464, 1204,  482, 2297,  5

In [81]:
#measurements.get_measurement_columns()

/lab/repos/CellProfiler/cellprofiler/measurement.py:1016: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(dtype, float):


[(u'Cell', u'AreaShape_Area', 'integer'),
 (u'Cell', u'AreaShape_Center_X', 'float'),
 (u'Cell', u'AreaShape_Center_Y', 'float'),
 (u'Cell', u'AreaShape_Center_Z', 'float'),
 (u'Cell', u'AreaShape_Compactness', 'float'),
 (u'Cell', u'AreaShape_Eccentricity', 'float'),
 (u'Cell', u'AreaShape_EulerNumber', 'float'),
 (u'Cell', u'AreaShape_Extent', 'float'),
 (u'Cell', u'AreaShape_FormFactor', 'float'),
 (u'Cell', u'AreaShape_MajorAxisLength', 'float'),
 (u'Cell', u'AreaShape_MaxFeretDiameter', 'float'),
 (u'Cell', u'AreaShape_MaximumRadius', 'float'),
 (u'Cell', u'AreaShape_MeanRadius', 'float'),
 (u'Cell', u'AreaShape_MedianRadius', 'float'),
 (u'Cell', u'AreaShape_MinFeretDiameter', 'float'),
 (u'Cell', u'AreaShape_MinorAxisLength', 'float'),
 (u'Cell', u'AreaShape_Orientation', 'float'),
 (u'Cell', u'AreaShape_Perimeter', 'float'),
 (u'Cell', u'AreaShape_Solidity', 'float'),
 (u'Cell', u'AreaShape_Zernike_0_0', 'float'),
 (u'Cell', u'AreaShape_Zernike_1_1', 'float'),
 (u'Cell', u'Area

In [6]:
image_pixel_data = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]], dtype=np.uint8)
image_obj_data = np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, 1, 1, 0, 0],
    [0, 2, 2, 1, 1, 0, 0],
    [0, 0, 2, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]], int)

In [7]:
IMAGE_NAME = 'TestImage'
OBJECT_NAME = 'TestObjects'
image = cellprofiler.image.Image(image_pixel_data, convert=False)
#image.pixel_data = image_pixel_data
image_set_list = cellprofiler.image.ImageSetList()
image_set = image_set_list.get_image_set(0)
image_set.add(IMAGE_NAME, image)

object_set = cellprofiler.object.ObjectSet()
objects = cellprofiler.object.Objects()
objects.segmented = image_obj_data
object_set.add_objects(objects, OBJECT_NAME)

module = cellprofiler.modules.measureobjectintensity.MeasureObjectIntensity()
module.images[0].name.value = IMAGE_NAME
module.objects[0].name.value = OBJECT_NAME

measurements = cellprofiler.measurement.Measurements()

ws = cellprofiler.workspace.Workspace(
    cellprofiler.pipeline.Pipeline(),
    module,
    image_set,
    object_set,
    measurements,
    image_set_list
)

In [62]:
ws.file_list.add_files_to_filelist(['file:///tmp/testfile.tif'])

In [64]:
ws.file_list.get_filelist()

[u'file:///tmp/testfile.tif']

In [8]:
image.pixel_data

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 2, 2, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [9]:
module.run(ws)

/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:539: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  np.issubdtype(hdf5_type, int) or
/lab/repos/CellProfiler/cellprofiler/utilities/hdf5_dict.py:541: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  hdf5_type_is_float = np.issubdtype(hdf5_type, float)


In [10]:
for feature_name in cellprofiler.modules.measureobjectintensity.ALL_MEASUREMENTS:
    feature_name = "%s_%s_%s" % (cellprofiler.modules.measureobjectintensity.INTENSITY, feature_name, IMAGE_NAME)
    data = measurements.get_current_measurement(OBJECT_NAME, feature_name)
    print(feature_name, data)

('Intensity_IntegratedIntensity_TestImage', array([6., 8.]))
('Intensity_MeanIntensity_TestImage', array([1., 2.]))
('Intensity_StdIntensity_TestImage', array([0., 0.]))
('Intensity_MinIntensity_TestImage', array([1., 2.]))
('Intensity_MaxIntensity_TestImage', array([1., 2.]))
('Intensity_IntegratedIntensityEdge_TestImage', array([6., 8.]))
('Intensity_MeanIntensityEdge_TestImage', array([1., 2.]))
('Intensity_StdIntensityEdge_TestImage', array([0., 0.]))
('Intensity_MinIntensityEdge_TestImage', array([1., 2.]))
('Intensity_MaxIntensityEdge_TestImage', array([1., 2.]))
('Intensity_MassDisplacement_TestImage', array([0., 0.]))
('Intensity_LowerQuartileIntensity_TestImage', array([1., 2.]))
('Intensity_MedianIntensity_TestImage', array([1., 2.]))
('Intensity_MADIntensity_TestImage', array([0., 0.]))
('Intensity_UpperQuartileIntensity_TestImage', array([1., 2.]))


In [6]:
pipeline.modules()

In [9]:
from cellprofiler.modules import measureobjectintensity

### CLI Testing

In [73]:
#!/opt/conda/envs/cellprofiler/bin/python /lab/repos/cytokit/python/external/cellprofiler/cpcli.py --help

In [ ]:
python /lab/repos/cytokit/python/external/cellprofiler/cpcli.py \
--config-file=/lab/data/20180101_codex_spleen/20180101_codex_mouse_spleen_balbc_slide1/output/v02/config/experiment.yaml \
--output-dir=/lab/data/20180101_codex_spleen/20180101_codex_mouse_spleen_balbc_slide1/output/v02 \
--log-level=DEBUG

To symlink locally for CPA:

```
cd /
sudo mkdir /lab
sudo chown eczech:staff /lab
cd /lab
ln -s /Volumes/disk1/ data
```